In [54]:
from sklearn.neighbors import NearestNeighbors
from torch.utils.data import Dataset, DataLoader
import torch
import numpy as np
import glob
import os
device='cuda' if torch.cuda.is_available() else 'cpu'
split_seed=0
split_ratio = .5
##for my architecture i dont trust batch size over one. but i think functionally it works fine. but it doesnt suit my purposes. that is why there is achtung. use it as you please with caution that
#I never have used batch size more than one.

#batch size of one = a batch of a whole protein. 

class HoodDS_from_npz(Dataset):
    """Generate hoods around n_neighbors, a hyperparameter. This choice determines how many n_nearest neighbors
    the egnn encoder sees.
    
    This function takes as input the pdb.npz numpy files and returns an """
    def __init__(self, paths, n_neighbors, keep_ids=False): #keep ids doesnt work
        self.data=[]; self.keep_ids = keep_ids; 
        nbr=NearestNeighbors(n_neighbors=n_neighbors,algorithm='brute')
        for p in paths:
            try:
                d=np.load(p,allow_pickle=True)
                if len(d['sites'])==0: continue
                nbr.fit(d['pos']); idx=nbr.kneighbors(d['sites'],return_distance=False)
                self.data.append((torch.from_numpy(d['z'][idx]),
                                  torch.from_numpy(d['pos'][idx]),
                                  torch.from_numpy(d['pks'])))
                #self.ids.append(os.path.splitext(os.path.basename(p))[0])
            except Exception as e: print("skip",p,e)
    def __len__(self): return len(self.data)
    def __getitem__(self,i):
        z,p,y=self.data[i]; 
        return z,p,y,self.ids[i] if self.keep_ids else z,p,y
    def pad(batch,k,device):
        #ids=[b[3] for b in batch] if keep_ids else None
        B=len(batch); S=max(b[0].shape[0] for b in batch)
        zt=torch.zeros(B,S,k,dtype=torch.int32,device=device)
        pt=torch.zeros(B,S,k,3,dtype=torch.float32,device=device)
        yt=torch.full((B,S),float('nan'),device=device); mt=torch.zeros(B,S,dtype=torch.bool,device=device)
        for b,data in enumerate(batch):
            z,p,y=data[0],data[1],data[2]
            s=z.shape[0]; zt[b,:s]=z; pt[b,:s]=p; yt[b,:s]=y; mt[b,:s]=True
        return (zt,pt,yt,mt) #if keep_ids else (zt,pt,yt,mt)

    def split(paths):
        """deterministic and random #TODO tunable determinism"""
        rng=np.random.RandomState(split_seed)
        idx=rng.permutation(len(paths)); cut=int(len(paths)*split_ratio)
        return [paths[i] for i in idx[:cut]], [paths[i] for i in idx[cut:]]
    
    
def run_hoodDS(n_neighbors, INPUTS_DIR, num_paths):
    if num_paths=="all":
        num_paths=len(glob.glob(INPUTS_DIR))
    coll = lambda b: HoodDS_from_npz.pad(b,n_neighbors,device)
    tr,val=HoodDS_from_npz.split(sorted(glob.glob(INPUTS_DIR))[:num_paths])
    train_ds=HoodDS_from_npz(tr, n_neighbors); val_ds=HoodDS_from_npz(val,
                                                               n_neighbors)

    tr_loader=DataLoader(train_ds,batch_size=1,shuffle=True,collate_fn=coll)#ACTHUNG: i dont trust batch size over 1 with this pad fn and ''  use AT OWN RISK.
    val_loader=DataLoader(val_ds,batch_size=1,shuffle=False,collate_fn=coll) #i dont trust batch size over 1 with this pad fn and personally dont need it use at OWN RISK
    return tr_loader, val_loader

INPUTS_DIR="../../data/pkegnn_INS/inputs/*.npz"
tr_loader, va_loader = run_hoodDS(100,INPUTS_DIR,3)



In [56]:
for a in tr_loader:
    print(a)

(tensor([[[ 8,  6,  6,  ...,  6,  7,  6],
         [ 8,  6,  8,  ...,  8,  6,  8],
         [ 8,  6,  8,  ...,  7,  6,  6],
         ...,
         [ 8,  6,  8,  ...,  6,  8,  8],
         [ 8,  6,  6,  ...,  6,  8,  6],
         [ 8,  6,  8,  ..., 16,  8,  7]]], dtype=torch.int32), tensor([[[[68.0350, 91.7610, 38.6070],
          [67.7180, 90.4250, 38.6710],
          [66.6170, 89.9510, 37.9820],
          ...,
          [66.3630, 93.9870, 31.3450],
          [66.6350, 85.7460, 33.8150],
          [74.7650, 95.7110, 37.5800]],

         [[73.6860, 76.4210, 42.9290],
          [73.1790, 76.1190, 41.8240],
          [73.8710, 75.8170, 40.8250],
          ...,
          [68.5280, 69.6540, 45.4050],
          [75.9200, 70.7350, 36.4920],
          [70.8710, 77.9780, 34.6550]],

         [[77.3800, 78.2300, 20.2870],
          [76.2080, 78.4030, 19.8910],
          [75.7660, 79.5170, 19.5510],
          ...,
          [83.1160, 82.4470, 14.3880],
          [82.7650, 76.5190, 27.6060],
     

In [63]:
for z,x,y,m,*_ in tr_loader:
    v=m.view(-1); z=z.view(-1,z.size(2))[v].to(device)
    x=x.view(-1,x.size(2),3)[v].to(device); y=y.view(-1)[v].to(device)
    print(z.shape)
    print(x.shape)
    print(y.shape)

torch.Size([126, 100])
torch.Size([126, 100, 3])
torch.Size([126])


In [ ]:
def run_model(loader):
    for z,x,y,m,*_ in loader:
        v=m.view(-1); z=z.view(-1,z.size(2))[v].to(device)
        x=x.view(-1,x.size(2),3)[v].to(device); y=y.view(-1)[v].to(device)
        #return z,x,y #Zs,coors,PK Shifts.
        #here you put a model.
        #preds = loop(x,z)
        #loss(preds, y)
run_model(tr_loader)
run_model(va_loader)